### Convert from synthetic profiles to interpolated xarray

The Argo data is in nc files for each individual profile, and each profile has a slightly different vertical grid. This non-uniform and non-standard grid is inconvenient to work with. Here we interpolate Argo data onto a uniform and standard grid for ease later. 

We also save all the data into a single nc file, to save time opening thousands of files. 

Some information on using profile data is here: https://argo.ucsd.edu/data/how-to-use-argo-files/

In [1]:
import numpy as np
import xarray as xr
import gsw 
import pandas as pd
import matplotlib.pyplot as plt 
import os
import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.interpolate import PchipInterpolator
from tqdm.notebook import tqdm

%matplotlib inline

In [45]:
data_dir = '/Users/dhruvbalwada/OneDrive/sogos_data/data/raw/Argo_synthetic/Profiles/'
#files = os.listdir(data_dir,'S*.nc')
files_SD = glob.glob(data_dir+'SD*.nc') # Delayed mode
files_SR = glob.glob(data_dir+'SR*.nc') # Real time

In [46]:
nfiles_SD = len(files_SD)
nfiles_SR = len(files_SR)

In [47]:
nfiles_SD, nfiles_SR

(55746, 7407)

#### Save the locations of the profiles

In [9]:
ds_locs_SD = xr.Dataset()

for count, file in tqdm(enumerate(files_SD)):
    
    # run a counter
#    if np.mod(count, 1000)==0:
#        print(count)
                             
    ds = xr.open_dataset(file)
    ds = ds.isel(N_PROF=0)
    
    ds_int = xr.Dataset()
    
    ds_int['JULD'] = ds.JULD
    ds_int['LATITUDE'] = ds.LATITUDE
    ds_int['LONGITUDE'] = ds.LONGITUDE
    ds_int['PLATFORM_NUM'] = ds.PLATFORM_NUMBER

    ds_locs_SD = xr.concat([ds_locs_SD, ds_int], dim='N_PROF')

In [10]:
ds_locs_SR = xr.Dataset()

for count, file in tqdm(enumerate(files_SR)):
    
    # run a counter
#    if np.mod(count, 1000)==0:
#        print(count)
                             
    ds = xr.open_dataset(file)
    ds = ds.isel(N_PROF=0)
    
    ds_int = xr.Dataset()
    
    ds_int['JULD'] = ds.JULD
    ds_int['LATITUDE'] = ds.LATITUDE
    ds_int['LONGITUDE'] = ds.LONGITUDE
    ds_int['PLATFORM_NUM'] = ds.PLATFORM_NUMBER

    ds_locs_SR = xr.concat([ds_locs_SR, ds_int], dim='N_PROF')

In [14]:
ds_locs_SR.to_netcdf('/Users/dhruvbalwada/OneDrive/sogos_data/data/processed/SO_Argo_locations_SR_25_may_2021.nc')

In [15]:
ds_locs_SD.to_netcdf('/Users/dhruvbalwada/OneDrive/sogos_data/data/processed/SO_Argo_locations_SD_25_may_2021.nc')

#### Look at some individual profiles to see what the data looks like

In [16]:
dsD = xr.open_dataset('/Users/dhruvbalwada/work_root/sogos/data/raw/Argo/Profiles/SD5903260_054.nc')
dsR = xr.open_dataset('/Users/dhruvbalwada/work_root/sogos/data/raw/Argo/Profiles/SR5905062_187.nc')

In [17]:
dsD

<xarray.Dataset>
Dimensions:                       (N_CALIB: 1, N_LEVELS: 70, N_PARAM: 4, N_PROF: 1)
Dimensions without coordinates: N_CALIB, N_LEVELS, N_PARAM, N_PROF
Data variables:
    DATA_TYPE                     object ...
    FORMAT_VERSION                object ...
    HANDBOOK_VERSION              object ...
    REFERENCE_DATE_TIME           object ...
    DATE_CREATION                 object ...
    DATE_UPDATE                   object ...
    PLATFORM_NUMBER               (N_PROF) object ...
    PROJECT_NAME                  (N_PROF) object ...
    PI_NAME                       (N_PROF) object ...
    STATION_PARAMETERS            (N_PROF, N_PARAM) object ...
    CYCLE_NUMBER                  (N_PROF) float64 ...
    DIRECTION                     (N_PROF) object ...
    DATA_CENTRE                   (N_PROF) object ...
    PARAMETER_DATA_MODE           (N_PROF, N_PARAM) object ...
    PLATFORM_TYPE                 (N_PROF) object ...
    FLOAT_SERIAL_NO               (N_PROF) object ...
    FIRMWARE_VERSION              (N_PROF) object ...
    WMO_INST_TYPE                 (N_PROF) object ...
    JULD                          (N_PROF) datetime64[ns] ...
    JULD_QC                       (N_PROF) object ...
    JULD_LOCATION                 (N_PROF) datetime64[ns] ...
    LATITUDE                      (N_PROF) float64 ...
    LONGITUDE                     (N_PROF) float64 ...
    POSITION_QC                   (N_PROF) object ...
    POSITIONING_SYSTEM            (N_PROF) object ...
    CONFIG_MISSION_NUMBER         (N_PROF) float64 ...
    PARAMETER                     (N_PROF, N_CALIB, N_PARAM) object ...
    SCIENTIFIC_CALIB_EQUATION     (N_PROF, N_CALIB, N_PARAM) object ...
    SCIENTIFIC_CALIB_COEFFICIENT  (N_PROF, N_CALIB, N_PARAM) object ...
    SCIENTIFIC_CALIB_COMMENT      (N_PROF, N_CALIB, N_PARAM) object ...
    SCIENTIFIC_CALIB_DATE         (N_PROF, N_CALIB, N_PARAM) object ...
    PROFILE_PRES_QC               (N_PROF) object ...
    PROFILE_TEMP_QC               (N_PROF) object ...
    PROFILE_PSAL_QC               (N_PROF) object ...
    PROFILE_DOXY_QC               (N_PROF) object ...
    PRES                          (N_PROF, N_LEVELS) float32 ...
    PRES_QC                       (N_PROF, N_LEVELS) object ...
    PRES_ADJUSTED                 (N_PROF, N_LEVELS) float32 ...
    PRES_ADJUSTED_QC              (N_PROF, N_LEVELS) object ...
    PRES_ADJUSTED_ERROR           (N_PROF, N_LEVELS) float32 ...
    TEMP                          (N_PROF, N_LEVELS) float32 ...
    TEMP_QC                       (N_PROF, N_LEVELS) object ...
    TEMP_dPRES                    (N_PROF, N_LEVELS) float32 ...
    TEMP_ADJUSTED                 (N_PROF, N_LEVELS) float32 ...
    TEMP_ADJUSTED_QC              (N_PROF, N_LEVELS) object ...
    TEMP_ADJUSTED_ERROR           (N_PROF, N_LEVELS) float32 ...
    PSAL                          (N_PROF, N_LEVELS) float32 ...
    PSAL_QC                       (N_PROF, N_LEVELS) object ...
    PSAL_dPRES                    (N_PROF, N_LEVELS) float32 ...
    PSAL_ADJUSTED                 (N_PROF, N_LEVELS) float32 ...
    PSAL_ADJUSTED_QC              (N_PROF, N_LEVELS) object ...
    PSAL_ADJUSTED_ERROR           (N_PROF, N_LEVELS) float32 ...
    DOXY                          (N_PROF, N_LEVELS) float32 ...
    DOXY_QC                       (N_PROF, N_LEVELS) object ...
    DOXY_dPRES                    (N_PROF, N_LEVELS) float32 ...
    DOXY_ADJUSTED                 (N_PROF, N_LEVELS) float32 ...
    DOXY_ADJUSTED_QC              (N_PROF, N_LEVELS) object ...
    DOXY_ADJUSTED_ERROR           (N_PROF, N_LEVELS) float32 ...
Attributes:
    title:                Argo float vertical profile
    institution:          CSIRO
    source:               Argo float
    history:              2020-11-01T07:28:14Z creation (software version 1.1...
    references:           http://www.argodatamgt.org/Documentation
    user_manual_version:  1.0
    Conventions:          Argo-3.1 CF-1.6
    featur

In [19]:
dsR

<xarray.Dataset>
Dimensions:                       (N_CALIB: 1, N_LEVELS: 329, N_PARAM: 4, N_PROF: 1)
Dimensions without coordinates: N_CALIB, N_LEVELS, N_PARAM, N_PROF
Data variables:
    DATA_TYPE                     object ...
    FORMAT_VERSION                object ...
    HANDBOOK_VERSION              object ...
    REFERENCE_DATE_TIME           object ...
    DATE_CREATION                 object ...
    DATE_UPDATE                   object ...
    PLATFORM_NUMBER               (N_PROF) object ...
    PROJECT_NAME                  (N_PROF) object ...
    PI_NAME                       (N_PROF) object ...
    STATION_PARAMETERS            (N_PROF, N_PARAM) object ...
    CYCLE_NUMBER                  (N_PROF) float64 ...
    DIRECTION                     (N_PROF) object ...
    DATA_CENTRE                   (N_PROF) object ...
    PARAMETER_DATA_MODE           (N_PROF, N_PARAM) object ...
    PLATFORM_TYPE                 (N_PROF) object ...
    FLOAT_SERIAL_NO               (N_PROF) object ...
    FIRMWARE_VERSION              (N_PROF) object ...
    WMO_INST_TYPE                 (N_PROF) object ...
    JULD                          (N_PROF) datetime64[ns] ...
    JULD_QC                       (N_PROF) object ...
    JULD_LOCATION                 (N_PROF) datetime64[ns] ...
    LATITUDE                      (N_PROF) float64 ...
    LONGITUDE                     (N_PROF) float64 ...
    POSITION_QC                   (N_PROF) object ...
    POSITIONING_SYSTEM            (N_PROF) object ...
    CONFIG_MISSION_NUMBER         (N_PROF) float64 ...
    PARAMETER                     (N_PROF, N_CALIB, N_PARAM) object ...
    SCIENTIFIC_CALIB_EQUATION     (N_PROF, N_CALIB, N_PARAM) object ...
    SCIENTIFIC_CALIB_COEFFICIENT  (N_PROF, N_CALIB, N_PARAM) object ...
    SCIENTIFIC_CALIB_COMMENT      (N_PROF, N_CALIB, N_PARAM) object ...
    SCIENTIFIC_CALIB_DATE         (N_PROF, N_CALIB, N_PARAM) object ...
    PROFILE_PRES_QC               (N_PROF) object ...
    PROFILE_TEMP_QC               (N_PROF) object ...
    PROFILE_PSAL_QC               (N_PROF) object ...
    PROFILE_DOXY_QC               (N_PROF) object ...
    PRES                          (N_PROF, N_LEVELS) float32 ...
    PRES_QC                       (N_PROF, N_LEVELS) object ...
    PRES_ADJUSTED                 (N_PROF, N_LEVELS) float32 ...
    PRES_ADJUSTED_QC              (N_PROF, N_LEVELS) object ...
    PRES_ADJUSTED_ERROR           (N_PROF, N_LEVELS) float32 ...
    TEMP                          (N_PROF, N_LEVELS) float32 ...
    TEMP_QC                       (N_PROF, N_LEVELS) object ...
    TEMP_dPRES                    (N_PROF, N_LEVELS) float32 ...
    TEMP_ADJUSTED                 (N_PROF, N_LEVELS) float32 ...
    TEMP_ADJUSTED_QC              (N_PROF, N_LEVELS) object ...
    TEMP_ADJUSTED_ERROR           (N_PROF, N_LEVELS) float32 ...
    PSAL                          (N_PROF, N_LEVELS) float32 ...
    PSAL_QC                       (N_PROF, N_LEVELS) object ...
    PSAL_dPRES                    (N_PROF, N_LEVELS) float32 ...
    PSAL_ADJUSTED                 (N_PROF, N_LEVELS) float32 ...
    PSAL_ADJUSTED_QC              (N_PROF, N_LEVELS) object ...
    PSAL_ADJUSTED_ERROR           (N_PROF, N_LEVELS) float32 ...
    DOXY                          (N_PROF, N_LEVELS) float32 ...
    DOXY_QC                       (N_PROF, N_LEVELS) object ...
    DOXY_dPRES                    (N_PROF, N_LEVELS) float32 ...
    DOXY_ADJUSTED                 (N_PROF, N_LEVELS) float32 ...
    DOXY_ADJUSTED_QC              (N_PROF, N_LEVELS) object ...
    DOXY_ADJUSTED_ERROR           (N_PROF, N_LEVELS) float32 ...
Attributes:
    title:                Argo float vertical profile
    institution:          JMA
    source:               Argo float
    history:              2020-10-29T13:03:11Z creation (software version 1.1...
    references:           http://www.argodatamgt.org/Documentation
    user_manual_version:  1.0
    Conventions:          Argo-3.1 CF-1.6
    feature

### This is where we open all files and interpolate the data we are doing to use.

We do this separately for the SD and SR files, so we can decide to merge or keep them separate as necessary. 

This is the stage at which a lot more QCing can be included. At present all that is done is:
- Use only the ADJUSTED data.
- Make sure there are more than 3 non-nan points in the O2, T, and S profiles. 
    - I assumed that bad data has been naned out in ADJUSTED profiles, but this needs to be checked. 
    - It should be checked if this will need to be done better by paying attention to the QC flags. 
    - Check why some profile files don't have adjusted data. (Can we get more data?)
    - What hapened to the files with O2 that Ken pointed out. 
- Interpolation is done using Pchip method.
- No extrapolation is done, only interpolation. 


In [84]:
%%time

## For the SD data
ds_final_SD = xr.Dataset()

std_levels = np.linspace(0,2000,1001) # Levels to do the interpolation to

files_more_profs_SD = []  # for profile files that have more than one profile
files_no_adjusted_SD = [] # for profile files that are missing the "ADJUSTED" variables
files_less_data_SD = []   # for profiles that did not have enough non-nan measurements

for count, file in tqdm(enumerate(files_SD)):
                             
    ds = xr.open_dataset(file)
    #take note if some files have more than one profiles
    if len(ds.N_PROF)>1:
        files_more_profs_SD = files_more_profs_SD + [file[-16:]]
        
    # for now only use the first one 
    # I find the presense of multiple N_PROF a bit confusing, since 
    ds = ds.isel(N_PROF=0)
    
    
    # remove nans
    if hasattr(ds, 'DOXY_ADJUSTED'):
        ds_int = xr.Dataset()
    
        ds_int['JULD'] = ds.JULD
        ds_int['LATITUDE'] = ds.LATITUDE
        ds_int['LONGITUDE'] = ds.LONGITUDE
        ds_int['PLATFORM_NUM'] = ds.PLATFORM_NUMBER
        
        # Here we do a check to make sure that the data is non-nan
        # A lot more testing, choices based on QC flags should be incoporated in the future. 
        O2 = ds.DOXY_ADJUSTED.where(~np.isnan(ds.DOXY_ADJUSTED), drop=True)
        P_O2= ds.PRES_ADJUSTED.where(~np.isnan(ds.DOXY_ADJUSTED), drop=True)
        
        
        T = ds.TEMP_ADJUSTED.where(~np.isnan(ds.TEMP_ADJUSTED), drop=True)
        P_T= ds.PRES_ADJUSTED.where(~np.isnan(ds.TEMP_ADJUSTED), drop=True)
        
        
        S = ds.PSAL_ADJUSTED.where(~np.isnan(ds.PSAL_ADJUSTED), drop=True)
        P_S= ds.PRES_ADJUSTED.where(~np.isnan(ds.PSAL_ADJUSTED), drop=True)
        
        
    else:
        files_no_adjusted_SD = files_no_adjusted_SD + [file[-16:]]
        continue

    # Check to make sure that there are some good data points
    if len(P_O2)>3 and len(P_T)>3 and len(P_S)>3:
        
        # get rid of repeated pressures
        O2 = O2.groupby(P_O2).mean()
        P_O2 = P_O2.groupby(P_O2).mean()

        T = T.groupby(P_T).mean()
        P_T = P_T.groupby(P_T).mean()

        S = S.groupby(P_S).mean()
        P_S = P_S.groupby(P_S).mean()

        # Record max pressure values
        #ds_int['P_O2max']= P_O2.max()
        #ds_int['P_Tmax']= P_T.max()
        #ds_int['P_Smax']= P_S.max()
        
        # interpolate in pressure
        f_O2 = PchipInterpolator(P_O2, O2,extrapolate=False)
        f_T  = PchipInterpolator(P_T, T,extrapolate=False)
        f_S = PchipInterpolator(P_S, S,extrapolate=False)

        O2_int = f_O2(std_levels)
        T_int = f_T(std_levels)
        S_int = f_S(std_levels)

        # not a good idea to change variable name, can cause confusion later.
        ds_int['TEMP'] = xr.DataArray(T_int, dims=['PRES'], 
                                      coords={'PRES':std_levels})
        ds_int['PSAL'] = xr.DataArray(S_int, dims=['PRES'], 
                                      coords={'PRES':std_levels})
        ds_int['DOXY'] = xr.DataArray(O2_int, dims=['PRES'], 
                                      coords={'PRES':std_levels})

        ds_final_SD = xr.concat([ds_final_SD, ds_int], dim='N_PROF')
        
    else:
        files_less_data_SD = files_less_data_SD + [file[-16:]]


CPU times: user 9h 20min 35s, sys: 2h 55min 43s, total: 12h 16min 18s
Wall time: 12h 41min 25s


In [85]:
ds_final_SD

<xarray.Dataset>
Dimensions:       (N_PROF: 46305, PRES: 1001)
Coordinates:
  * PRES          (PRES) float64 0.0 2.0 4.0 6.0 ... 1.996e+03 1.998e+03 2e+03
Dimensions without coordinates: N_PROF
Data variables:
    PLATFORM_NUM  (N_PROF) object b'5903260 ' b'5903256 ' ... b'5903613 '
    PSAL          (N_PROF, PRES) float64 nan nan nan 34.21 ... 34.66 nan nan
    DOXY          (N_PROF, PRES) float64 nan nan nan 297.2 ... 234.4 nan nan
    LONGITUDE     (N_PROF) float64 179.5 -142.1 90.27 ... -9.28 -74.91 -24.53
    TEMP          (N_PROF, PRES) float64 nan nan nan 6.43 ... -0.09784 nan nan
    LATITUDE      (N_PROF) float64 -51.38 -52.35 -59.59 ... -61.24 -66.42 -66.57
    JULD          (N_PROF) datetime64[ns] 2011-08-16T03:44:22.000004864 ... 2014-11-03T19:23:23.002071808

In [95]:
# write the data
with open('/Users/dhruvbalwada/OneDrive/sogos_data/data/processed/files_less_data_SD.txt', 'w') as filehandle:
    for listitem in files_less_data_SD:
        filehandle.write('%s\n' % listitem)

In [96]:
# write the data 
with open('/Users/dhruvbalwada/OneDrive/sogos_data/data/processed/files_no_adjusted_SD.txt', 'w') as filehandle:
    for listitem in files_no_adjusted_SD:
        filehandle.write('%s\n' % listitem)

In [89]:
ds_final_SD.to_netcdf('/Users/dhruvbalwada/OneDrive/sogos_data/data/processed/Argo_SD_int_oxygen_25_may_2021.nc')

In [90]:
%%time

## For the SR data 
ds_final_SR = xr.Dataset()

std_levels = np.linspace(0,2000,1001) # Levels to do the interpolation to

files_more_profs_SR = []
files_no_adjusted_SR = []
files_less_data_SR = []

for count, file in tqdm(enumerate(files_SR)):
                             
    ds = xr.open_dataset(file)
    #take note if some files have more than one profiles
    if len(ds.N_PROF)>1:
        files_more_profs_SR = files_more_profs_SR + [file[-16:]]
        
    # for now only use the first one 
    # I find the presense of multiple N_PROF a bit confusing 
    ds = ds.isel(N_PROF=0)
    
    
    # remove nans
    if hasattr(ds, 'DOXY_ADJUSTED'):
        ds_int = xr.Dataset()
    
        ds_int['JULD'] = ds.JULD
        ds_int['LATITUDE'] = ds.LATITUDE
        ds_int['LONGITUDE'] = ds.LONGITUDE
        ds_int['PLATFORM_NUM'] = ds.PLATFORM_NUMBER
        
        # Here we do a check to make sure that the data is non-nan
        # A lot more testing, choices based on QC flags should be incoporated in the future. 
        O2 = ds.DOXY_ADJUSTED.where(~np.isnan(ds.DOXY_ADJUSTED), drop=True)
        P_O2= ds.PRES_ADJUSTED.where(~np.isnan(ds.DOXY_ADJUSTED), drop=True)
        
        
        T = ds.TEMP_ADJUSTED.where(~np.isnan(ds.TEMP_ADJUSTED), drop=True)
        P_T= ds.PRES_ADJUSTED.where(~np.isnan(ds.TEMP_ADJUSTED), drop=True)
        
        
        S = ds.PSAL_ADJUSTED.where(~np.isnan(ds.PSAL_ADJUSTED), drop=True)
        P_S= ds.PRES_ADJUSTED.where(~np.isnan(ds.PSAL_ADJUSTED), drop=True)
        
        
    else:
        files_no_adjusted_SR = files_no_adjusted_SR + [file[-16:]]
        continue

    # Check to make sure that there are some good data points
    if len(P_O2)>3 and len(P_T)>3 and len(P_S)>3:
        
        # get rid of repeated pressures
        O2 = O2.groupby(P_O2).mean()
        P_O2 = P_O2.groupby(P_O2).mean()

        T = T.groupby(P_T).mean()
        P_T = P_T.groupby(P_T).mean()

        S = S.groupby(P_S).mean()
        P_S = P_S.groupby(P_S).mean()

        # Record max pressure values
        #ds_int['P_O2max']= P_O2.max()
        #ds_int['P_Tmax']= P_T.max()
        #ds_int['P_Smax']= P_S.max()
        
        # interpolate in pressure
        f_O2 = PchipInterpolator(P_O2, O2,extrapolate=False)
        f_T  = PchipInterpolator(P_T, T,extrapolate=False)
        f_S = PchipInterpolator(P_S, S,extrapolate=False)

        O2_int = f_O2(std_levels)
        T_int = f_T(std_levels)
        S_int = f_S(std_levels)

        ds_int['TEMP'] = xr.DataArray(T_int, dims=['PRES'], 
                                      coords={'PRES':std_levels})
        ds_int['PSAL'] = xr.DataArray(S_int, dims=['PRES'], 
                                      coords={'PRES':std_levels})
        ds_int['DOXY'] = xr.DataArray(O2_int, dims=['PRES'], 
                                      coords={'PRES':std_levels})

        ds_final_SR = xr.concat([ds_final_SR, ds_int], dim='N_PROF')
        
    else:
        files_less_data_SR = files_less_data_SR + [file[-16:]]


CPU times: user 14min 29s, sys: 17.6 s, total: 14min 47s
Wall time: 15min 1s


In [92]:
ds_final_SR

<xarray.Dataset>
Dimensions:       (N_PROF: 1366, PRES: 1001)
Coordinates:
  * PRES          (PRES) float64 0.0 2.0 4.0 6.0 ... 1.996e+03 1.998e+03 2e+03
Dimensions without coordinates: N_PROF
Data variables:
    PLATFORM_NUM  (N_PROF) object b'2902736 ' b'2902736 ' ... b'5904982 '
    PSAL          (N_PROF, PRES) float64 nan nan nan 33.88 ... nan nan nan nan
    DOXY          (N_PROF, PRES) float64 nan nan nan 291.3 ... nan nan nan nan
    LONGITUDE     (N_PROF) float64 119.2 128.4 48.03 ... 39.59 -26.75 -12.3
    TEMP          (N_PROF, PRES) float64 nan nan nan 8.255 ... nan nan nan nan
    LATITUDE      (N_PROF) float64 -47.46 -50.29 -58.45 ... -51.66 -45.45 -44.2
    JULD          (N_PROF) datetime64[ns] 2016-03-03T12:04:02.999999488 ... 2020-11-08T04:45:49.000494336

In [98]:
with open('/Users/dhruvbalwada/OneDrive/sogos_data/data/processed/files_less_data_SR.txt', 'w') as filehandle:
    for listitem in files_less_data_SR:
        filehandle.write('%s\n' % listitem)

In [99]:
with open('/Users/dhruvbalwada/OneDrive/sogos_data/data/processed/files_no_adjusted_SR.txt', 'w') as filehandle:
    for listitem in files_no_adjusted_SR:
        filehandle.write('%s\n' % listitem)

In [91]:
ds_final_SR.to_netcdf('/Users/dhruvbalwada/OneDrive/sogos_data/data/processed/Argo_SR_int_oxygen_25_may_2021.nc')